In [1]:
from ctypes import *
from ctypes import util
from decimal import Decimal
from enum import Enum
import math
import unittest

In [2]:
try:
    import platform
    OSX_VERSION = tuple(int(v) for v in platform.mac_ver()[0].split('.')[:2])
except:
    OSX_VERSION = None

In [3]:
from rubicon.objc import ObjCClass, objc_method, objc_classmethod, objc_property, NSEdgeInsets, NSEdgeInsetsMake

Library was compiled using make in the project root folder and then copied in /usr/local/lib

In [4]:
harnesslib = util.find_library('rubiconharness')
if harnesslib is None:
    raise RuntimeError("Couldn't load Rubicon test harness library. Have you set DYLD_LIBRARY_PATH?")
cdll.LoadLibrary(harnesslib)

<CDLL '/usr/local/lib/librubiconharness.dylib', handle 104c00bc0 at 0x104e65320>

In [5]:
def test_field():
    print ("A field on an instance can be accessed and mutated")
    Example = ObjCClass('Example')
    
    obj = Example.alloc().init()
    
    print (obj.baseIntField, 22)
    print (obj.intField, 33)
    
    obj.baseIntField = 8888
    obj.intField = 9999
    
    print (obj.baseIntField, 8888)
    print (obj.intField, 9999)
test_field()

A field on an instance can be accessed and mutated
22 22
33 33
8888 8888
9999 9999


In [6]:
def test_method():
    print ("An instance method can be invoked.")
    Example = ObjCClass('Example')

    obj = Example.alloc().init()

    print (obj.accessBaseIntField(), 22)
    print (obj.accessIntField(), 33)

    obj.mutateBaseIntFieldWithValue_(8888)
    obj.mutateIntFieldWithValue_(9999)

    print (obj.accessBaseIntField(), 8888)
    print (obj.accessIntField(), 9999)
test_method()

An instance method can be invoked.
22 22
33 33
8888 8888
9999 9999


In [7]:
def test_static_field():
    print ("A static field on a class can be accessed and mutated")
    Example = ObjCClass('Example')

    Example.mutateStaticBaseIntFieldWithValue_(1)
    Example.mutateStaticIntFieldWithValue_(11)

    print(Example.staticBaseIntField, 1)
    print(Example.staticIntField, 11)

    Example.staticBaseIntField = 1188
    Example.staticIntField = 1199

    print(Example.staticBaseIntField, 1188)
    print(Example.staticIntField, 1199)
test_static_field()

A static field on a class can be accessed and mutated
1 1
11 11
1188 1188
1199 1199


In [8]:
def test_static_method():
    print ("A static method on a class can be invoked.")
    Example = ObjCClass('Example')

    Example.mutateStaticBaseIntFieldWithValue_(2288)
    Example.mutateStaticIntFieldWithValue_(2299)

    print(Example.accessStaticBaseIntField(), 2288)
    print(Example.accessStaticIntField(), 2299)
test_static_method()

A static method on a class can be invoked.
2288 2288
2299 2299


In [9]:
def test_mutator_like_method():
    print ("A method that looks like a mutator doesn't confuse issues.")
    Example = ObjCClass('Example')

    obj1 = Example.alloc().init()

    # setSpecialValue: looks like it might be a mutator
    # for a specialValue property, but this property doesn't exist.

    # We can invoke the method directly...
    obj1.setSpecialValue_(42)

    # ... but retrieving like a property is an error
    # with self.assertRaises(AttributeError):
    try:
        obj1.specialValue
    except AttributeError:
        print ('AttributeError caught')

    # ...until you set it explicitly...
    obj1.specialValue = 37

    # ...at which point it's fair game to be retrieved.
    print(obj1.specialValue, 37)
test_mutator_like_method()

A method that looks like a mutator doesn't confuse issues.
AttributeError caught
37 37


In [10]:
def test_non_existent_class():
    print ("A Name Error is raised if a class doesn't exist.")

    # If a class doesn't exist, raise NameError
    #with self.assertRaises(NameError):
    try:
        ObjCClass('DoesNotExist')
    except NameError:
        print('NameError caught')

    # If you try to create a class directly from a pointer, and
    # the pointer isn't valid, raise an error.
    #with self.assertRaises(RuntimeError):
    try:
        ObjCClass(0)
    except RuntimeError:
        print ('RuntimeError caught')
test_non_existent_class()

A Name Error is raised if a class doesn't exist.
NameError caught
RuntimeError caught


In [11]:
def test_non_existent_field():
    print ("An attribute error is raised if you invoke a non-existent field.")
    Example = ObjCClass('Example')

    obj1 = Example.alloc().init()

    # Non-existent fields raise an error.
    #with self.assertRaises(AttributeError):
    try:
        obj1.field_doesnt_exist
    except AttributeError:
        print ('AttributeError caught')

    # Cache warming doesn't affect anything.
    #with self.assertRaises(AttributeError):
    try:
        obj1.field_doesnt_exist
    except AttributeError:
        print ('AttributeError caught')
test_non_existent_field()

An attribute error is raised if you invoke a non-existent field.
AttributeError caught
AttributeError caught


In [12]:
def test_non_existent_method():
    print ("An attribute error is raised if you invoke a non-existent method.")
    Example = ObjCClass('Example')

    obj1 = Example.alloc().init()

    # Non-existent methods raise an error.
    #with self.assertRaises(AttributeError):
    try:
        obj1.method_doesnt_exist()
    except AttributeError:
        print ('AttributeError caught')

    # Cache warming doesn't affect anything.
    #with self.assertRaises(AttributeError):
    try:
        obj1.method_doesnt_exist()
    except AttributeError:
        print ('AttributeError caught')
test_non_existent_method()

An attribute error is raised if you invoke a non-existent method.
AttributeError caught
AttributeError caught


In [13]:
def test_non_existent_static_field():
    print("An attribute error is raised if you invoke a non-existent static field.")
    Example = ObjCClass('Example')

    # Non-existent fields raise an error.
    #with self.assertRaises(AttributeError):
    try:
        Example.static_field_doesnt_exist
    except AttributeError:
        print ('AttributeError caught')

    # Cache warming doesn't affect anything.
    #with self.assertRaises(AttributeError):
    try:
        Example.static_field_doesnt_exist
    except AttributeError:
        print ('AttributeError caught')
test_non_existent_static_field()

An attribute error is raised if you invoke a non-existent static field.
AttributeError caught
AttributeError caught


In [14]:
def test_non_existent_static_method():
    print("An attribute error is raised if you invoke a non-existent static method.")
    Example = ObjCClass('Example')

    # Non-existent methods raise an error.
    #with self.assertRaises(AttributeError):
    try:
        Example.static_method_doesnt_exist()
    except AttributeError:
        print ('AttributeError caught')

    # Cache warming doesn't affect anything.
    #with self.assertRaises(AttributeError):
    try:
        Example.static_method_doesnt_exist()
    except AttributeError:
        print ('AttributeError caught')
test_non_existent_static_method()

An attribute error is raised if you invoke a non-existent static method.
AttributeError caught
AttributeError caught


In [15]:
def test_polymorphic_constructor():
    print("Check that the right constructor is activated based on arguments used")
    Example = ObjCClass('Example')

    obj1 = Example.alloc().init()
    obj2 = Example.alloc().initWithIntValue_(2242)
    obj3 = Example.alloc().initWithBaseIntValue_intValue_(3342, 3337)

    print(obj1.baseIntField, 22)
    print(obj1.intField, 33)

    print(obj2.baseIntField, 44)
    print(obj2.intField, 2242)

    print(obj3.baseIntField, 3342)
    print(obj3.intField, 3337)

    # Protected constructors can't be invoked
    #with self.assertRaises(AttributeError):
    try:
        Example.alloc().initWithString_("Hello")
    except AttributeError:
        print ('AttributeError caught')
test_polymorphic_constructor()

Check that the right constructor is activated based on arguments used
22 22
33 33
44 44
2242 2242
3342 3342
3337 3337
AttributeError caught


In [16]:
def test_static_access_non_static():
    print("An instance field/method cannot be accessed from the static context")
    Example = ObjCClass('Example')

    obj = Example.alloc().init()

    #with self.assertRaises(AttributeError):
    try:
        obj.staticIntField
    except AttributeError:
        print ('AttributeError caught')

    #with self.assertRaises(AttributeError):
    try:
        obj.get_staticIntField()
    except AttributeError:
        print ('AttributeError caught')
test_static_access_non_static()

An instance field/method cannot be accessed from the static context
AttributeError caught
AttributeError caught


In [17]:
def test_non_static_access_static():
    print("A static field/method cannot be accessed from an instance context")
    Example = ObjCClass('Example')

    #with self.assertRaises(AttributeError):
    try:
        Example.intField
    except AttributeError:
        print ('AttributeError caught')

    #with self.assertRaises(AttributeError):
    try:
        Example.accessIntField()
    except AttributeError:
        print ('AttributeError caught')
test_non_static_access_static()

A static field/method cannot be accessed from an instance context
AttributeError caught
AttributeError caught


In [18]:
def test_string_argument():
    print("A method with a string argument can be passed.")
    Example = ObjCClass('Example')
    example = Example.alloc().init()
    print(example.duplicateString_("Wagga"), "WaggaWagga")
test_string_argument()

A method with a string argument can be passed.
WaggaWagga WaggaWagga


In [19]:
def test_enum_argument():
    print("An enumerated type can be used as an argument.")
    Example = ObjCClass('Example')

    obj = Example.alloc().init()

    print(obj.accessBaseIntField(), 22)
    print(obj.accessIntField(), 33)

    class MyEnum(Enum):
        value1 = 8888
        value2 = 9999
        value3 = 3333
        value4 = 4444

    obj.mutateBaseIntFieldWithValue_(MyEnum.value1)
    obj.mutateIntFieldWithValue_(MyEnum.value2)

    print(obj.accessBaseIntField(), MyEnum.value1.value)
    print(obj.accessIntField(), MyEnum.value2.value)

    obj.baseIntField = MyEnum.value3
    obj.intField = MyEnum.value4

    print(obj.accessBaseIntField(), MyEnum.value3.value)
    print(obj.accessIntField(), MyEnum.value4.value)
test_enum_argument()

An enumerated type can be used as an argument.
22 22
33 33
8888 8888
9999 9999
3333 3333
4444 4444


In [20]:
def test_string_return():
    print("If a method or field returns a string, you get a Python string back")
    Example = ObjCClass('Example')
    example = Example.alloc().init()
    print(example.toString(), "This is an ObjC Example object")
test_string_return()

If a method or field returns a string, you get a Python string back
This is an ObjC Example object This is an ObjC Example object


In [21]:
def test_constant_string_return():
    print("If a method or field returns a *constant* string, you get a Python string back")
    Example = ObjCClass('Example')
    example = Example.alloc().init()
    print(example.smiley(), "%-)")
test_constant_string_return()

If a method or field returns a *constant* string, you get a Python string back
%-) %-)


In [22]:
def test_number_return():
    print("If a method or field returns a NSNumber, it is converted back to native types")
    Example = ObjCClass('Example')
    example = Example.alloc().init()

    print(example.theAnswer(), 42)
    print(example.twopi(), 2.0 * math.pi, 5)
test_number_return()

If a method or field returns a NSNumber, it is converted back to native types
42 42
6.2831854820251465 6.283185307179586 5


In [23]:
def test_float_method():
    print("A method with a float arguments can be handled.")
    Example = ObjCClass('Example')
    example = Example.alloc().init()
    print(example.areaOfSquare_(1.5), 2.25)
test_float_method()

A method with a float arguments can be handled.
2.25 2.25


In [24]:
def test_double_method():
    print("A method with a double arguments can be handled.")
    Example = ObjCClass('Example')
    example = Example.alloc().init()
    print(example.areaOfCircle_(1.5), 1.5 * math.pi, 5)
test_double_method()

A method with a double arguments can be handled.
4.71238898038469 4.71238898038469 5


In [25]:
def test_decimal_method():
    print("A method with a NSDecimalNumber arguments can be handled.")
    Example = ObjCClass('Example')
    example = Example.alloc().init()

    result = example.areaOfTriangleWithWidth_andHeight_(Decimal('3.0'), Decimal('4.0'))
    print(result, Decimal('6.0'))
    print(isinstance(result, Decimal), 'Result should be a Decimal')
test_decimal_method()

A method with a NSDecimalNumber arguments can be handled.
6 6.0
True Result should be a Decimal


In [26]:
def test_object_return():
    print("If a method or field returns an object, you get an instance of that type returned")
    Example = ObjCClass('Example')
    example = Example.alloc().init()

    Thing = ObjCClass('Thing')
    thing = Thing.alloc().initWithName_value_('This is thing', 2)

    example.thing = thing

    the_thing = example.thing
    print(the_thing.toString(), "This is thing 2")
test_object_return()

If a method or field returns an object, you get an instance of that type returned
This is thing 2 This is thing 2


In [27]:
def test_duplicate_class_registration():
    print("If you define a class name twice in the same runtime, you get an error.")

    NSObject = ObjCClass('NSObject')

    # First definition should work.
    class MyClass(NSObject):
        pass

    # Second definition will raise an error.
    # Without protection, this is a segfault.
    #with self.assertRaises(RuntimeError):
    try:
        class MyClass(NSObject):
            pass
    except RuntimeError:
        print ('RuntimeError caught')
test_duplicate_class_registration()

If you define a class name twice in the same runtime, you get an error.
RuntimeError caught


In [28]:
def test_interface():
    print("An ObjC protocol implementation can be defined in Python.")

    results = {}

    NSObject = ObjCClass('NSObject')

    class Handler(NSObject):
        @objc_method
        def initWithValue_(self, value: int):
            self.value = value
            return self

        @objc_method
        def peek_withValue_(self, example, value: int) -> None:
            results['string'] = example.toString() + " peeked"
            results['int'] = value + self.value

        @objc_method
        def poke_withValue_(self, example, value: int) -> None:
            results['string'] = example.toString() + " poked"
            results['int'] = value + self.value

        @objc_method
        def reverse_(self, input):
            return ''.join(reversed(input))

        @objc_method
        def message(self):
            return "Alea iacta est."

        @objc_classmethod
        def fiddle_(cls, value: int) -> None:
            results['string'] = "Fiddled with it"
            results['int'] = value

    # Create two handler instances so we can check the right one
    # is being invoked.
    handler1 = Handler.alloc().initWithValue_(5)
    handler2 = Handler.alloc().initWithValue_(10)

    # Create an Example object, and register a handler with it.
    Example = ObjCClass('Example')
    example = Example.alloc().init()
    example.callback = handler2

    # Check some Python-side attributes
    print(handler1.value, 5)
    print(handler2.value, 10)

    # Invoke the callback; check that the results have been peeked as expected
    example.testPeek_(42)

    print(results['string'], 'This is an ObjC Example object peeked')
    print(results['int'], 52)

    example.testPoke_(37)

    print(results['string'], 'This is an ObjC Example object poked')
    print(results['int'], 47)

    print(example.getMessage() + ' should be: Alea iacta est.')

    print(example.reverseIt_('Alea iacta est.') + ' should be: .tse atcai aelA')

    Handler.fiddle_(99)

    print(results['string'], 'Fiddled with it')
    print(results['int'], 99)
test_interface()

An ObjC protocol implementation can be defined in Python.
5 5
10 10
This is an ObjC Example object peeked This is an ObjC Example object peeked
52 52
This is an ObjC Example object poked This is an ObjC Example object poked
47 47
Alea iacta est. should be: Alea iacta est.
.tse atcai aelA should be: .tse atcai aelA
Fiddled with it Fiddled with it
99 99


In [29]:
def test_class_properties():
    print("A Python class can have ObjC properties with synthezied getters and setters.")

    NSObject = ObjCClass('NSObject')
    NSURL = ObjCClass('NSURL')

    class URLBox(NSObject):

        # takes no type: All properties are pointers
        url = objc_property()

        @objc_method
        def getSchemeIfPresent(self):
            if self.url is not None:
                return self.url.scheme
            return None

    box = URLBox.alloc().init()

    # Default property value is None
    if box.url == None:
        print('box.url is none')

    # Assign an object via synthesized property setter and call method that uses synthesized property getter
    url = NSURL.alloc().initWithString_('https://www.google.com')
    box.url = url
    print(box.getSchemeIfPresent() + ' should be https')

    # Assign None to dealloc property and see if method returns expected None
    box.url = None
    if box.getSchemeIfPresent() == None:
        print('box.getSchemeIfPresent() is None')
test_class_properties()

A Python class can have ObjC properties with synthezied getters and setters.
box.url is none
https should be https
box.getSchemeIfPresent() is None


In [30]:
def test_function_NSEdgeInsetsMake():
    print("Python can invoke NSEdgeInsetsMake to create NSEdgeInsets.")

    insets = NSEdgeInsets(0.0, 1.1, 2.2, 3.3)
    other_insets = NSEdgeInsetsMake(0.0, 1.1, 2.2, 3.3)

    # structs are NOT equal
    print(insets, other_insets)

    # but their values are
    print(insets.top, other_insets.top)
    print(insets.left, other_insets.left)
    print(insets.bottom, other_insets.bottom)
    print(insets.right, other_insets.right)
test_function_NSEdgeInsetsMake()

Python can invoke NSEdgeInsetsMake to create NSEdgeInsets.
<rubicon.objc.types.NSEdgeInsets object at 0x104ebea60> <rubicon.objc.types.NSEdgeInsets object at 0x104ebeae8>
0.0 0.0
1.1 1.1
2.2 2.2
3.3 3.3
